<a href="https://colab.research.google.com/github/michalis0/MGT-502-Data-Science-and-Machine-Learning/blob/main/03_Classification/Lab_4_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import to load arff file from url
from scipy.io import arff
import urllib.request
import io

# Sklearn import
from sklearn.model_selection import train_test_split # Splitting the data set
from sklearn.preprocessing import MinMaxScaler, StandardScaler # Normalization and standard scaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Label and 1-hot encoding
from sklearn.linear_model import LogisticRegression # Logistic regression model
from sklearn.linear_model import LogisticRegressionCV # Logistic regression with cross-validation
from sklearn.metrics import accuracy_score  # Accuracy
from sklearn.metrics import confusion_matrix # Confusion matrix
from sklearn.metrics import precision_score, recall_score, f1_score  # Precision, recall, and f1 score

# Lab 4. Classification 1

## Content

The goal of this walkthrough is to provide you with first insights on classification, focusing on Logistic Regression. After presenting the main concepts, you will be introduced to the techniques to implement the algorithms in Python. Finally, it will be your turn to practice doing the first part of Assignment 3.

This notebook is organized as follows:
- [Background](#Background)
    - [Objective](#Objective)
    - [Examples of classification](#Examples-of-classification)
    - [Logistic regression model](#Logistic-regression-model)
        - [Logistic Loss function](#Logistic-Loss-function)
- [Implementation](#Implementation)
    - [Load and discover dataset](#Load-and-discover-dataset)
    - [Splitting the dataset](#Splitting-the-dataset)
    - [Rescaling](#Rescaling)
    - [Building and training our classifier](#Building-and-training-our-classifier)
    - [Using the classifier to make prediction](#Using-the-classifier-to-make-prediction)
    - [Evaluating our classifier](#Evaluating-our-classifier)
        - [Accuracy](#Accuracy)
        - [Confusion matrix](#Confusion-matrix)
        - [Precision and Recall](#Precision-and-Recall)
   - [Adding cross-validation](#Adding-cross-validation)
   - [Visualizing our model: Decision boundary](#Visualizing-our-model:-Decision-boundary)

## Background

### Objective

We now move from regression to another important branch of machine learning: classification. As the name indicates, the idea is to classify items of a dataset into **predefined classes** for which labelled data is already available. Therefore, classification is an instance of supervised machine learning.

Classification is similar to regression, but instead of predicting a continuous target, classification algorithms attempt to apply a discrete number of labels or classes to each observation.

While in a regression the targets are generally numerical and continuous-valued, oftentimes the targets in classification are categorical.  

However, classification can be applied in settings where the target is numerical. For example, we might want to predict whether the unemployment rate for a country will be low, medium, or high, but do not care about the actual number. In such cases, determining the "optimal" way to categorize our target variable might require some creativity, and it is important to properly justify our assumptions.

Finally, note that many problems can be written either as classification or regression. Hence, many ML algorithms have variants that perform regression or classification tasks.

### Examples of classification

- Image classification: e.g., is this a cat or not?
- Audio classification: identify bird species from birdsong
- Labeling emails: is it a spam or not?
- Risky or safe loan application: should a bank provide a loan to applicant or not?
- Prediction of customer behaviour: will a customer buy this new product or not?  
- Prediction of economic performance of a country: will there be a recession or not?  
- etc.

### Logistic regression model

Suppose we have *n* observations of an outcome $\boldsymbol{y}$ and *d* associated features $\boldsymbol{x_1}$, $\boldsymbol{x_2}$, ... , $\boldsymbol{x_d}$ (note that $\boldsymbol{y}$, $\boldsymbol{x_1}$, ..., $\boldsymbol{x_d}$ are vectors):

| | Outcome | Feature 1 | Feature 2 | ... | Feature d |
|:-------|:----------:|:----------:|:----------:|:----------:|:----------:|
| Observation 1 | $y_1$ | $x_{11}$ | $x_{12}$ | ... | $x_{1d}$ |
| Observation 2 | $y_2$ | $x_{21}$ | $x_{22}$ | ... | $x_{2d}$ |
| ... | ... | ... | ... | ... | ... |
| Observation n | $y_n$ | $x_{n1}$ | $x_{n2}$ | ... | $x_{nd}$ |

We will focus on binary classification for now. In other words, our outcome can take two values, 0 and 1, which represent two classes (e.g., cat or dog, spam email or not, risky or safe loan, etc.).

Remember when we did multivariate linear regression, we assumed that our model function $f_{\text{mv}}$, i.e., our prediction, was a linear combination of our features. For each observation $i$, we assumed:
$$f_{\text{mv}}(\boldsymbol{X_{i*}}, \boldsymbol{w}):=w_0 + w_1 x_{i,1} +  w_2 x_{i,2} + ... +  w_d x_{i,d}$$
with $\boldsymbol{w}=(w_0, w_1, ..., w_d)$ the vector of weights, and $\boldsymbol{X}=[\boldsymbol{x_1}$, ... , $\boldsymbol{x_d}]$ the matrix of feature variables.

For each observation, our true outcome was $y_i = f_{\text{mv}}(\boldsymbol{X_{i*}}, \boldsymbol{w}) + \epsilon_i$, and our goal was to minimize the errors.

In this setting, our model function $f_{\text{mv}}$ can take any values. It is thus suited when our outcome is continuous. However, with binary classification, we are dealing with discrete values, and more precisely with 0 and 1. How can we modify our model to obtain better prediction?

The idea of logistic regression is to transform the predictions obtained with a linear regression such that the predictions are between 0 and 1. To do so, we rely on the [Sigmoid (logistic) function](https://en.wikipedia.org/wiki/Sigmoid_function):

$$S(x) = \frac{1}{1 + e^{-x}}$$

<center>
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1280px-Logistic-curve.svg.png' width="300">
</center>

Source: Qef, from [Wikipedia Logistic Curve plot](https://commons.wikimedia.org/wiki/File:Logistic-curve.svg)

With logistic regression, we apply the sigmoid function to the output of the multivariate regression model. Let $f_{\text{logi}}$ be the prediction function of a logistic regression, we have:

$$f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}):= \frac{1}{1 + e^{-(w_0 + w_1 x_{i,1} +  w_2 x_{i,2} + ... +  w_d x_{i,d})}}$$

$f_{\text{logi}}$ represents the probability that a given observation belongs to class 1, i.e., $y_i=1$:
- We predict that the observation belongs to class 1 when $f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}) \geq 0.5$, i.e., when $w_0 + w_1 x_{i,1} +  w_2 x_{i,2} + ... +  w_d x_{i,d} \geq 0$;
- Reciprocally, we predict that the observation belongs to class 0 when $f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w})<0.5$, i.e., $w_0 + w_1 x_{i,1} +  w_2 x_{i,2} + ... +  w_d x_{i,d}<0$.

Now our problem is the same as before: we want to minimize the errors of our model, learning the weights $w_0$, $w_1$, ..., $w_d$ from our data. To do so, we are minimizing our loss function... but which one? We will explore one option below.

*Note*: In the case of multiclass logistic regression, the idea is the same: we need to transform the output of our multivariate model. However, instead of using the sigmoid function, we are applying the [Softmax function](https://en.wikipedia.org/wiki/Softmax_function), a generalization of the sigmoid function.  

#### Logistic Loss function

For linear regression, we used the Least Squared Error as loss function:

$$ \min_\boldsymbol{w} \sum_{i=1}^n (y_i - f_{\text{mv}}(\boldsymbol{X_{i*}}, \boldsymbol{w}))^2 $$

Can we use the same for logistic regression? No! Indeed, using Least Squared Error with our new prediction function $f_{\text{mv}}$ would result in a non-convex graph, which is not ideal for our minimization problem since we could be stuck in local minima:

<center>
<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*3o9_XoQP4TaceYPRZVHlxg.png' width="300">
</center>

Source: Issam Laradji, [Non-convex Optimization](https://www.cs.ubc.ca/labs/lci/mlrg/slides/non_convex_optimization.pdf)

So which loss function can we use? Ideally, we want to assign more punishment when predicting 1 while the actual value is 0 and when predicting 0 while the actual value is 1. One such function is the... **Logistic Loss**:

$$L(\boldsymbol{y}, \boldsymbol{X}, \boldsymbol{w})= -\frac{1}{n} \sum_{i=1}^n [y_i \log(f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w})) + (1-y_i) \log(1-f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}))] $$

Let's decompose our function to understand a bit more how it works. For each observation $i$, the cost is:

$$\text{Cost}_i = - y_i \log(f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w})) - (1-y_i) \log(1-f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}))$$

- When $y_i = 1$, $\text{Cost}_i = - \log(f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}))$. Hence, if our predicted probability is 1, we have $\text{Cost}_i=0$, i.e., no cost. However, when our predicted probability is approaching 0, our cost goes to infinity (because the logarithm goes to minus infinity when we get closer to zero).
- When $y_i = 0$, $\text{Cost}_i = - \log(1-f_{\text{logi}}(\boldsymbol{X_{i*}}, \boldsymbol{w}))$, and it works the other way around. If our predicted probability is zero, the cost is zero, but when our predicted probability is approaching 1, our cost goes to infinity.

<center>
<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*_NeTem-yeZ8Pr9cVUoi_HA.png' width="400">
</center>

Source: Shuyu Luo, [Loss Function (Part II): Logistic Regression](https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11)

The Logistic Loss not only punishes errors with a very large cost, it is also convex. Hence, we can still apply Gradient Descent, Newton's Method, and other optimization algorithms!

To learn more:
- [Loss Function (Part II): Logistic Regression](https://towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11), by Shuyu Luo, Published in Towards Data Science
- [Understanding the log loss function](https://medium.com/analytics-vidhya/understanding-the-loss-function-of-logistic-regression-ac1eec2838ce), by Susmith Reddy, Published in Analytics Vidhya


## Implementation

We are already familiar with the **sklearn** library, when we implemented regression algorithms last week ([Documentation](https://scikit-learn.org/stable/index.html)). We will keep on using this library to implement a logistic regression.

For the walkthough we will use a **dataset on wine quality**. As usual, you can find it in the /data folder.

<img src='https://assets.pbimgs.com/pbimgs/rk/images/dp/wcm/202145/0002/schott-zwiesel-classico-wine-glasses-c.jpg' width="300">

The wine data set consists of 11 different parameters of wine such as alcohol content, acidity, and pH, which were measured for several wine samples from the North of Portugal.

Source: P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In *Decision Support Systems*, Elsevier, 47(4):547-553, 2009. Dataset obtained from UCI Machine Learning repository, [Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality).

These wines were derived from different cultivars; therefore there have different quality, as a score between 0 and 10. We grouped the wines into two quality classes: 0 and 1, representing respectively  "poor quality" (score 0-5), and "good quality" (score 6-10).

Our goal here is to find a model that can predict the class of wine given the 11 measured parameters, and find out the major differences among the two classes.

Ok, let's go!

### Load and discover dataset

In [ ]:
#Load the dataset
url = "https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/wine-quality-red.csv"
wines = pd.read_csv(url).drop_duplicates().dropna() # drop duplicates and NaN values

# Display a sample of the data
display(wines.head())

# Print columns
print(wines.columns.values)

Note that we only have numerical variables, and thus won't need to encode categorical variables.

However, we will need to rescale our features since, for instance, chlorides values are lower than 1 while sulfur dioxide can attain a value of 289:

In [ ]:
wines.describe()

We now define our features - all wine parameters - and our target variable - the wine quality:

In [ ]:
# Define features and target variable
X = wines.drop(columns='quality')
y = wines['quality']

We know check how many observations we have for each class:

In [ ]:
# Count the number of observations (rows) corresponding to each value
y.value_counts()

We have 719 "good" wines and 640 "poor" quality wines. The number of observations for each class influence the quality of our predictions. Here, our dataset is reasonably balanced.

### Splitting the dataset

As always, the first step is to split our data into random training and test subsets. Recall that the training set is used to learn the parameters of our model while the test set is used to evaluate our predictions.

We use the `train_test_split` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)) of sklearn, imported with the following line of code (already done at the beginning of the notebook):

```python
from sklearn.model_selection import train_test_split
```

The test size here is of 25% of the original data.

In [ ]:
#Split data set into a train and a test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, shuffle=True)

print(f"The training set has {X_train.shape[0]} observations, and the test set has {X_test.shape[0]} observations.")

### Rescaling

When we have a dataset with features that have very distinct ranges, we might get biased results. We want the features to be in the same or similar range, which also helps the interpretation of the model parameters (weights).

We therefore **normalize** the data. It involves transforming all values for a specific attribute so that they fall within a small specified range. We can use `StandardScaler()`, ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)) `MinMaxScaler()` ([Documentation](https://scikit-learn.org/0.15/modules/generated/sklearn.preprocessing.MinMaxScaler.html)) or others for normalization.

In our example we will normalize both our **train AND test data** using `MinMaxScaler()`. Here is the import line:

```python
from sklearn.preprocessing import MinMaxScaler
```

For each observation $x_{ij}$, with $i$ the observation (row) and $j$ the feature (column), the MinMax Scaler applies the following transformation:  
$$x_{scaled,ij}=\frac{x_{ij} - \min(\boldsymbol{x_j})}{\max(\boldsymbol{x_j})-\min(\boldsymbol{x_j})}$$

**IMPORTANT**: When you normalize the train data, you need to do the same modification (here normalization) to the test data. In other words, you train your scaler on your training set, and apply the same transformation to the training and test set.

In [ ]:
# Define the scaler
scaler = MinMaxScaler()

# Fit the scaler
scaler.fit(X_train) # here the scaler learns the min and max of each attribute from the training set

# Transform the train and the test set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Note that the fit and transform steps can be merged into one for the training set:
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

### Building and training our classifier

To predict the class of our target variable we use a logistic regression. The sklearn module is called `LogisticRegression()` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)). Here is the import line:

```python
from sklearn.linear_model import LogisticRegression
```

Note that L2-regularization is applied by default. By specifying the argument *penalty*, you can specify the regularization techniques, namely 'l1', 'l2', 'elasticnet', or None.

You can also specify the *solver*. By default, 'lbfgs' is used, which stands for [Limited-memory Broyden–Fletcher–Goldfarb–Shanno](https://en.wikipedia.org/wiki/Limited-memory_BFGS).
Note that the choice of the algorithm depends on the penalty chosen. You can refer to the documentation for insights on the choice of solver/penalty depending on your problem and data.

**A short note on solver:**
L-BFGS approximates the Broyden–Fletcher–Goldfarb–Shanno algorithm ([BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm)), which is based on [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization), an alternative to Gradient Descent. While the Gradient Descent rely on the gradient (first-order derivatives) to update our parameters, Newton's method also makes use of the [Hessian matrix](https://en.wikipedia.org/wiki/Hessian_matrix), i.e., the second-order derivatives. Newton's method generally converges faster than Gradient Descent. However, Newton's method is computationally-expensive and the Hessian might not even exist. Hence, numerical methods called [Quasi-Newton](https://en.wikipedia.org/wiki/Quasi-Newton_method), such as BFGS, have been developed to solve optimization problems.  

In [ ]:
# 1. Set up our model
model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)

# 2. Fit our model
model.fit(X_train, y_train)

After fitting the model, we can easily retrieve the values of the different weights coefficients: the intercept with `.intercept_`, and the weights of each feature with `.coef_.flatten()`:

In [ ]:
# Dataframe with the intercept and coefficients (weights) of the logistic model
model_coeff = pd.DataFrame(np.concatenate((model.intercept_, model.coef_.flatten())),
                     index=["Intercept"] + list(X.columns.values),
                     columns=['Coefficients logistic model'])
model_coeff

It seems like the level of alcohol and volatile acidity were the most important features to predict the wine quality, at least in our model.

### Using the classifier to make prediction

Once our model has been trained, we can use `predict()` to predict new values. We predict the values from the test set to then evaluate the model, estimating the accuracy of our classifier.

In [ ]:
y_pred = model.predict(X_test)

We can even access the probabilities that one observation belongs to one class or the other with `predict_proba()`. The largest probability determines the predicted class.

In [ ]:
# Dataframe with probabilities that our first 5 observations belong to each class
model_proba = pd.DataFrame(model.predict_proba(X_test)[0:4],
                     columns=['Probability poor-quality wine', 'Probability good wine'])
model_proba

###  Evaluating our classifier

We will now evaluate the performance of our classifier using several metrics.

#### Accuracy

Perhaps the most intuitive classification metric is *accuracy*, which is the fraction of correct vs total predictions:

$$ \mathrm{Accuracy} = \frac{\# \mathrm{\, correct \, predictions}}{\# \mathrm{\, total \, predictions}} $$

For a sklearn classifier, this can be computed using the `score` method.

In [ ]:
# Accuracy on the test set
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(model.score(X_test, y_test)))

# Accuracy on the training set
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(model.score(X_train, y_train)))

Alternatively, we could use the `accuracy_score` module, imported with the following line of code:

```python
from sklearn.metrics import accuracy_score
```

In [ ]:
accuracy_test = accuracy_score(y_test, y_pred)

print(f'Accurary of Logistic regression classifier on test set: {accuracy_test :.2f}')

When the testing accuracy is much lower than the training accuracy, we have an overfitting issue. Reciprocally, when the testing accuracy is similar to or higher than the training accuracy, the model might be underfitting, and we could consider either using a more powerful model or adding additional
features.

Our testing accuracy is 72%. Is that good? It depends! The quality of our prediction depends on the distribution of class in our original data:

In [ ]:
y.value_counts().plot.bar(color=['purple', 'blue'], grid=False)
plt.ylabel('Number of observations')
plt.title('Number of observations of each class in the wine dataset');

Imagine we have a naive classifier that always predict the majority class. We call the default rate (or base rate) the accuracy of this classifier, which is equal to the size of the most common class over the size of the full dataset:

$$\text{Default rate} = \frac{\# \text{ most frequent class}}{\# \text{ total observations}}$$

If the default rate is too high, then the classification can be biased, meaning that the data set has too many observations of one class compared to the other classes, and has hence more impact on the classification results.

The accuracy of our classifier should be better than the default rate. Let's calculate this default rate!

In [ ]:
# Compute the default rate
quality_0 = wines.loc[wines["quality"] == 0].shape[0]
print('# occurrence of class 0: ', quality_0)
quality_1 = wines.loc[wines["quality"] == 1].shape[0]
print('# occurence of class 1: ', quality_1)
defaultrate = max(quality_0, quality_1)/(wines["quality"].shape[0])
print(f'Default rate = {defaultrate:0.4f}')

Our default rate is about 52.9% while our classifier accuracy is 72.4%. Not too bad!

#### Confusion matrix

The confusion matrix allows us to get more details on the performance of our model. It will allow us to see what our classification model is getting right and what types of errors it is making.

Here is how a confusion matrix looks like:

|   | Class 0 predicted  | Class 1 predicted  |    
|---|---|---|
| **Class 0 actual**  |  TN |FP   |    
| **Class 1 actual**  | FN  | TP  |   


where TP = true positive, FN = false negative, FP = false positive and TN = true negative. Here class 1 is considered the "True" class.

We are using the `confusion_matrix` module of sklearn, imported with the following line of code:

``` python
from sklearn.metrics import confusion_matrix
```

It requires as input the true values and the predicted values:

In [ ]:
confusion_matrix(y_test, y_pred)

To obtain a more visual representation, we will use `heatmap` from the `seaborn` library:

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix');

On the diagonal of the confusion matrix are our correct predictions, while the off-diagonal elements are incorrect predictions. We can thus quickly identify whether one class is driving down or up the accuracy results, or if the results are more balanced.

Recall that the accuracy is the number of correct predictions divided by the number of incorrect predictions. Thus, the accuracy is the sum of the diagonal elements of the confusion matrix, divided by the total number of elements:

In [ ]:
print('Accuracy on test set using sklearn: {:.6f}'.format(accuracy_score(y_test, y_pred)))
print('Accuracy on test set by hand: {:.6f}'.format((113+133)/(113+133+45+49)))

#### Precision and Recall

In many contexts, the accuracy would be an appropriate way to evaluate a model, but in others, this would be insufficient.

For example, suppose we want to use a classification model to predict the likelihood of someone having a rare, but serious health condition. If the condition is very rare (say it appears in 0.01% of the population), then a model that always predicts false would have 99.99% accuracy, but the false negatives could have large consequences.

To capture such situations, we often use two other very common metrics: the *precision* and *recall*.

Let class 1 represents our positive cases ("true" class), the confusion matrix is:

|   | Class 0 predicted  | Class 1 predicted  |    
|---|---|---|
| **Class 0 actual**  |  TN | FP   |    
| **Class 1 actual**  | FN  | TP  |

- *Precision*: The number of true positives over the number of positive  predictions. Precision tells us how often the model was correct when it predicted true.

$$\text{Precision} = \frac{\# \text{ true positives}}{\# \text{ predicted positives}} = \frac{ \text{TP}}{\text{TP+FP}}$$

- *Recall*: The number of true positives over the number of actual positives. Recall answers the question, “What fraction of the positives did we get correct?”  

$$\text{Recall} = \frac{\# \text{ true positives}}{\# \text{ actual positives}} = \frac{\text{TP}}{ \text{TP+FN}}$$

In many settings, both precision and recall are equally important and a compound metric known as the *F1-score* is used:

$$\text{F1} = 2 \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$$

The F1 score is bounded between 0 and 1. It will only achieve a value of 1 if both precision and recall are exactly 1.

We will compute the precision using `precision_score` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)), the recall using `recall_score` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)), and the F1 score using `f1_score` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)). Here are the import lines:

``` python
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
```

For a binary classifier, all metrics will report by default the scores associated with the positive class (i.e., with observations equal to 1). If we are interested in the results for another class, we can specify some parameters. For instance, the parameter `average = None` will return the scores of each class:

In [ ]:
print('The precision for class 1 (good wines) is: {:0.3f}'.format(precision_score(y_test, y_pred)))
print('The recall for class 1 is: {:0.3f}'.format(recall_score(y_test, y_pred)))
print('The F1 score for class 1 is: {:0.3f}'.format(f1_score(y_test, y_pred)))

In [ ]:
# Precision of each class
model_precision = precision_score(y_test, y_pred, average = None)
# Recall of each class
model_recall = recall_score(y_test, y_pred, average = None)
# F1 score of each class
model_f1 = f1_score(y_test, y_pred, average = None)

# Visualize all results in a dataframe:
model_eval = pd.DataFrame([model_precision, model_recall, model_f1],
                    index = ['Precision', 'Recall', 'F1 score'],
                    columns=['Class 0', 'Class 1'])
model_eval

Evaluating your classifier is key, and it depends on our objectives and on our data. You can find all the sklearn model evaluation metrics [here](https://scikit-learn.org/stable/modules/model_evaluation.html).

### Adding cross-validation

We will see how the performance of our model evolves when implementing cross-validation technique. We are using the `LogisticRegressionCV()` module ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV)), imported as follows:

```python
from sklearn.linear_model import LogisticRegressionCV
```

In [ ]:
# Set up our model
model_cv = LogisticRegressionCV(penalty='l2', solver='lbfgs', cv=5, max_iter=1000)

# Fit our model
model_cv.fit(X_train, y_train)

Let's check the accuracy of our new model and compare it to the previous one:

In [ ]:
# Accuracy on the test set
model_cv_test_accuracy = model_cv.score(X_test, y_test) # Cross-validation model
model_test_accuracy = model.score(X_test, y_test)       # Logistic regression
# Accuracy on the training set
model_cv_train_accuracy = model_cv.score(X_train, y_train) # Cross-validation model
model_train_accuracy = model.score(X_train, y_train)       # Logistic regression
# Gather results in a dataframe:
models_accuracy = [[model_cv_test_accuracy, model_test_accuracy],
                   [model_cv_train_accuracy, model_train_accuracy]]
model_compar = pd.DataFrame(models_accuracy,
                    index = ['Test accuracy', 'Train accuracy'],
                    columns=['Cross-validation model', 'Logistic regression'])
model_compar

The accuracy improved a bit. Let's have a look at the confusion matrix:

In [ ]:
# Confusion matrix
confusion_cv = confusion_matrix(y_test, model_cv.predict(X_test))
# Heatmap of confusion matrix
sns.heatmap(confusion_cv, annot=True, cmap='Blues', fmt='.4g')
plt.xlabel('Predicted label')
plt.ylabel('True labels')
plt.title('Confusion Matrix with Cross Validation');

We correctly predicted one additional observation from the class 0. Not a spectacular improvement but we'll take it!

### Visualizing our model: Decision boundary

Let's build a new classifier, only using two features, namely alcohol and volatile acidity. Our objective is to visualize the decision boundary, i.e., the line that splits our two classes.

In [ ]:
# Extract features of interest
X2 = X[['volatile acidity', 'alcohol']]

# Split data set into a train and a test data sets
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X2, y, test_size=0.25, random_state=0, shuffle=True)

# Set up our model
model_2 = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)

# Fit our model
model_2.fit(X_train_2, y_train_2)

# Compute accuracy
print('Accuracy of two-features classifier on test set: {:.2f}'
     .format(model_2.score(X_test_2, y_test_2)))
print('Accuracy of two-features classifier on training set: {:.2f}'
     .format(model_2.score(X_train_2, y_train_2)))

# Dataframe with the intercept and coefficients (weights) of the model
model_2_coeff = pd.DataFrame(np.concatenate((model_2.intercept_, model_2.coef_.flatten())),
                     index=["Intercept"] + list(X2.columns.values),
                     columns=['Coefficients 2-features model'])
model_2_coeff

We now plot the decision boundary.

Note that the decision boundary represents the points for which the probability to belong to class 0 is the same as the probability to belong to class 1. For such points, we have:

$$w_0 + w_1 x_{i1} + w_2 x_{i2} =0 $$

Reorganizing, we get:

$$x_{i2} = - \frac{w_0 + w_1 x_{i1}}{w_2}$$

In [ ]:
fig, ax = plt.subplots()

# Scatter plot of volatile acidity and alcohol, colored by their class
scatter = ax.scatter(X2['volatile acidity'].values,  # values of volatile acidity on x-axis
            X2['alcohol'].values,                    # values of alcohol on y-axis
            c=y,                   # color class by class
            edgecolors='k',        # black edge colors
            cmap=plt.cm.Paired)    # color of markers: Paired colormap

# Plot decision boundary
x_vals = np.array(ax.get_xlim())    # array with x-axis limits
y_vals = -(model_2.intercept_[0]+x_vals*model_2.coef_[0][0])/model_2.coef_[0][1]  # equation of decision boundary
plt.plot(x_vals, y_vals, '--', c="red")  # plot decision boundary in red color

# Add labels, title, and legend
plt.xlabel('Wine volatile acidity content ')
plt.ylabel('Wine alcohol content')
plt.title('Decision boundary for classification model using two features')
plt.legend(*scatter.legend_elements(),loc="lower right", title="Classes")

# Display plot
plt.show()

Let's interpret this graph. The red doted line shows the boundary between the two regions. Each observation above the line will belong to class 1, i.e., good wine, while each observation below the line will belong to class 0, i.e., poor quality wine. Furthermore, the closer a point is to the decision region boundary the more uncertain we are about the accuracy of the classification.

Let's test this, by creating two new points:
- point A will have a volatile acidity of 1.2 and an alcohol content of 11.5
- point B will have the same volatile acidity, but an alcohol content of 13

In [ ]:
fig, ax = plt.subplots()

# Scatter plot of volatile acidity and alcohol, colored by their class
scatter = ax.scatter(X2['volatile acidity'].values,  # values of volatile acidity on x-axis
            X2['alcohol'].values,                    # values of alcohol on y-axis
            c=y,                   # color class by class
            edgecolors='k',        # black edge colors
            cmap=plt.cm.Paired)    # color of markers: Paired colormap

# Plot decision boundary
x_vals = np.array(ax.get_xlim())    # array with x-axis limits
y_vals = -(model_2.intercept_[0]+x_vals*model_2.coef_[0][0])/model_2.coef_[0][1]  # equation of decision boundary
plt.plot(x_vals, y_vals, '--', c="red")  # plot decision boundary in red color

# Add new points
plt.scatter(1.2, 11.5, c='red') # Point A
plt.scatter(1.2, 13, c='red')   # Point B
plt.text(1.23, 11.4, 'Point A')  # Point A label
plt.text(1.23, 12.9, 'Point B')       # Point B label

# Add labels, title, and legend
plt.xlabel('Wine volatile acidity content ')
plt.ylabel('Wine alcohol content')
plt.title('Decision boundary for classification model using two features')
plt.legend(*scatter.legend_elements(),loc="lower right", title="Classes")

# Display plot
plt.show()

We predict the class of each point:

In [ ]:
# Dataframe with new points
x_new= pd.DataFrame([[1.2, 11.5], [1.2,13]], columns=['volatile acidity', 'alcohol'])

# Predicted class
print('Point A belongs to class {}.'.format(model_2.predict(x_new)[0]))
print('Point B belongs to class {}.'.format(model_2.predict(x_new)[1]))

Finally, we can check the probabilities to belong to each class:

In [ ]:
# Dataframe with probabilities to belong to each class
x_new_proba = pd.DataFrame(model_2.predict_proba(x_new),
                     index = ['A', 'B'],
                     columns=['Class 0', 'Class 1'])
x_new_proba

Point A is closer to the boundary decision line, hence, its probability to belong to class 0 (0.63) is lower than the probability of point B to belong to class 1 (0.76): the prediction is more uncertain.